### 파일 압축 풀기

In [1]:
import zipfile

filepath = r'C:\Users\kwon\project\project\dacon\resume\data.zip'
folder = r'C:\Users\kwon\project\project\dacon\resume\data'

with zipfile.ZipFile(filepath, 'r') as zip_ref :
    zip_ref.extractall(folder)

## EDA

In [2]:
import pandas as pd
import os

data = {}
data_path = r'C:\Users\kwon\project\project\dacon\resume\data'
for fname in os.listdir(data_path):
    if fname.endswith('.csv'):
        file_path = os.path.join(data_path, fname)
        data[fname.rsplit('.', 1)[0]] = pd.read_csv(file_path)


In [3]:
for key in data.keys():
    print(key)
    print(data[key].info())
    print('@@@@@@@@@@@@@@@@@@@@@')

apply_train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57946 entries, 0 to 57945
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   resume_seq       57946 non-null  object
 1   recruitment_seq  57946 non-null  object
dtypes: object(2)
memory usage: 905.5+ KB
None
@@@@@@@@@@@@@@@@@@@@@
company
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2377 entries, 0 to 2376
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   recruitment_seq   2377 non-null   object
 1   company_type_seq  2377 non-null   int64 
 2   supply_kind       2377 non-null   int64 
 3   employee          2377 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 74.4+ KB
None
@@@@@@@@@@@@@@@@@@@@@
recruitment
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6695 entries, 0 to 6694
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtyp

In [26]:
recru = data['recruitment']
recru.recruitment_seq.unique().size #6695개의 고유한 원서

recru.qualifications.unique()
recru.address_seq1.value_counts()

address_seq1
3.0     6490
5.0      174
20.0      17
11.0       5
1.0        4
13.0       2
9.0        1
2.0        1
Name: count, dtype: int64

In [31]:
cert = data["resume_certificate"]
cert.certificate_contents.head(10)
cert["resume_seq"].unique().size # 5976명만 

5976

In [38]:
from collections import Counter
counter = Counter(cert["resume_seq"])

most_common_element, most_common_count = counter.most_common(1)[0]
most_common_element, most_common_count 

cert[cert["resume_seq"] == 'U06784']

apply = data["apply_train"]
apply[apply["resume_seq"] == 'U06784']

,resume_seq,recruitment_seq
740,U06784,R06400
2745,U06784,R03834
3322,U06784,R04936
12144,U06784,R05992
15658,U06784,R06484
15999,U06784,R03898
16241,U06784,R02359
19743,U06784,R05329
20405,U06784,R02600
25607,U06784,R02113


## 협업 필터링

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

apply_train_df = data['apply_train']

# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
user_item_matrix[user_item_matrix > 1] = 1

# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_item_matrix)

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T



In [7]:
print(type(user_item_matrix))
print(user_item_matrix.info())
print(user_item_matrix.size)
print(user_item_matrix)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
Index: 8482 entries, U00001 to U08482
Columns: 6695 entries, R00001 to R06695
dtypes: int64(6695)
memory usage: 433.3+ MB
None
56786990
recruitment_seq  R00001  R00002  R00003  R00004  R00005  R00006  R00007  \
resume_seq                                                                
U00001                0       0       0       0       0       0       0   
U00002                0       0       0       0       0       0       0   
U00003                0       0       0       0       0       0       0   
U00004                0       0       0       1       0       0       0   
U00005                0       0       0       0       0       0       0   
...                 ...     ...     ...     ...     ...     ...     ...   
U08478                0       0       0       0       0       0       0   
U08479                0       0       0       0       0       0       0   
U08480                0       0       

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시 데이터
X = np.array([[1, 2, 3],
              [4, 5, 6]])
Y = np.array([[1, 2, 3],
              [4, 5, 7]])

# 코사인 유사도 계산
similarity_scores = cosine_similarity(X, Y)
print(similarity_scores)

[[1.         0.9860133 ]
 [0.97463185 0.99703746]]


### Prediction

In [9]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)
    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
    for job in recommended_jobs:
        recommendations.append([user, job])

In [11]:
user_item_matrix.index
user_item_matrix.loc['U00001'][user_item_matrix.loc['U00001'] == 1]

sorted_job_indices = user_predicted_scores[1].argsort()[::-1]

user_predicted_scores[2][user_predicted_scores[2].argsort()[::-1][:5]]

array([0.59022969, 0.56060945, 0.37491422, 0.21520452, 0.10181875])

### Submission

In [12]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./baseline_submit.csv', index=False)

## 아이템 기반 

### 데이터 합치기


In [30]:
data.keys()

dict_keys(['apply_train', 'company', 'recruitment', 'resume', 'resume_certificate', 'resume_education', 'resume_language', 'sample_submission'])

In [39]:
resume = data["resume"]
resume_lan = data["resume_language"]
print(len(resume_lan.resume_seq))
len(resume_lan.resume_seq.unique()) # 어학이 두개이상인 사람이 있음

869


820

##

In [54]:
from surprise import SVD, Reader, Dataset

# apply_train_df에서 지원 여부를 나타내는 컬럼이 'Applied'라고 가정하고, 지원했으면 1, 아니면 0의 값을 가집니다.
# Reader 객체 생성
reader = Reader(rating_scale=(0, 1))  # 0: 지원하지 않음, 1: 지원함

# 데이터 로드
data = Dataset.load_from_df(apply_train_df[['resume_seq', 'recruitment_seq', 'Applied']], reader)  # 'Applied'는 지원 여부를 나타내는 컬럼이라고 가정

# 전체 데이터셋으로 훈련
trainset = data.build_full_trainset()
svd = SVD()
svd.fit(trainset)

ModuleNotFoundError: No module named 'surprise'

## Implict 라이브러리

In [39]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

apply_train_df = data['apply_train']

# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
user_item_matrix[user_item_matrix > 1] = 1

In [46]:
import sys

print(sys.executable)

c:\Program Files\Python311\python.exe


In [48]:
import os
print(os.environ.get('VIRTUAL_ENV'))

None


In [42]:
import implicit
import numpy as np
from scipy.sparse import csr_matrix

# 데이터를 sparse matrix로 변환
user_item = csr_matrix(user_item_matrix)

# ALS 모델 생성 및 학습
model = implicit.als.AlternatingLeastSquares(factors=5, iterations=20)
model.fit(user_item.T)  # Implicit에서는 항목-사용자 행렬을 기대합니다. 그래서 전치해줍니다.

# 특정 사용자(예: 사용자 0)에 대한 추천 항목 가져오기
user_id = 0
recommended = model.recommend(user_id, user_item)

print(recommended)

ModuleNotFoundError: No module named 'implicit'